In [ ]:
from zmq.sugar.socket import T
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from google.colab import drive

drive.mount ('/content/drive')

col_name = ["prix", "titre", "location", "addresse", "excerpt", "description", "size", "details", "property_overview"] #columns name

df = pd.DataFrame (columns = col_name) #data frame for storing houses data

url = 'https://www.property24.com/for-sale/cape-town/western-cape/432?PropertyCategory=House%2cApartmentOrFlat%2cTownhouse' #url of the main page (page 1)

i = 1

while (1):
    """Looping through pages of the website"""
    try:
        page = urlopen (url) # trying to open the page
    ##Catching exceptions
    except HTTPError :
        print ("Page not found")
        break
    except URLError :
        print ("Server not found")
        break

    html = bs (page, 'html.parser') #parsing the brute page

    try:
        content = html.body.find ('div', class_="col-9").find ('div', class_="js_listingResultsContainer") #extracting contents which interest us
        houses = content.find_all (class_="p24_content") # then taking houses container
    except AttributeError:
        break

    for house in houses :
        """Looping through all houses """
        lg = [] #initializing a list for storying the i-th line of the dataframe
        try:
            price = house.find (class_="p24_price").get_text ()
            price = re.sub (r'\s', '', str(price))
        except AttributeError:
            price = None
        lg.append (price)


        try:
            title = house.find (class_="p24_title").get_text ()
            title = re.sub (r'[\t\n\r]', '', str(title))
        except AttributeError:
            title = None
        lg.append (title)

        try:
            location = house.find (class_="p24_location").get_text ()
            location = re.sub (r'[\t\n\r]', '', str(location))
        except AttributeError:
            location = None
        lg.append (location)

        try:
            address = house.find (class_="p24_address").get_text ()
            address = re.sub (r'[\t\n\r]', '', str(address))
        except AttributeError:
            address = None
        lg.append (address)

        try:
            excerpt = house.find (class_="p24_excerpt").get_text ()
            excerpt = re.sub (r'[\t\n\r]', '', str(excerpt))
        except AttributeError:
            excerpt = None
        lg.append (excerpt)

        try:
            description = house.find (class_="p24_description").get_text ()
            description = re.sub (r'[\t\n\r]', '', str(description))
        except AttributeError:
            description = None
        lg.append (description)

        try:
            size = house.find (class_="p24_size").span.get_text()
            name = house.find (class_="p24_size").get("title")
            size = name + " " + size
        except AttributeError:
            size = None
        lg.append (size)

        details = house.find_all (class_="p24_featureDetails")
        text = ""
        for det in details :
            try:
                feature = det.get ("title")
                feature = re.sub (r'[\t\n\r]', '', str(feature))
                number = det.find ('span').get_text ()
                number = re.sub (r'[\t\n\r]', '', str(number))
                text += number + " " + feature + ","
            except AttributeError:
                text = None
            text_clean = ""
            if text != None :
                for i in range(len(text)-1):
                    text_clean += text[i]
        lg.append (text_clean)
        try:
            linkin = house.parent.get ("href")
            linkin = "https://www.property24.com"+linkin
            inside = urlopen (linkin)
            inside_content = bs (inside.read(), 'html.parser')

            fourth = inside_content.find ('div', class_="panel-body")
            fourthcontent = fourth.find_all ('div', class_="row p24_propertyOverviewRow")
            text = ""
            for e in fourthcontent :
                name = e.find ('div', class_="col-6 p24_propertyOverviewKey").get_text ()
                info = e.find ('div', class_="p24_info").get_text ()
                text = name + " : " + info + "\n"
            lg.append (text)
        except TypeError:
              lg.append (None)


        df.loc[len(df)] = lg
        print (df.loc[len(df)-1])
        print (f"\n")
    i += 1
    url='https://www.property24.com/for-sale/cape-town/western-cape/432'+'/p'+str(i)+'?PropertyCategory=House%2cApartmentOrFlat%2cTownhouse'

df.to_excel ('./capetown.xlsx', index=False)

Mounted at /content/drive
prix                                                R1100000
titre                                                   None
location                                          Muizenberg
addresse                                  1 St Davids Street
excerpt                                                 None
description                2 Bedroom Apartment in Muizenberg
size                                        Floor Size 62 m²
details              2 Bedrooms,1 Bathrooms,1 Parking Spaces
property_overview                                           
Name: 0, dtype: object


prix                                                         R39000000
titre                                                 11 Bedroom House
location                                                     Camps Bay
addresse                                                          None
excerpt                          Nestled in the heart of Camps Bay,...
description                                  